# JapDic

## Questions for testownik generator

Version 1.0, made by Vik

In this code I want to create simple quizes to learn words for japanese A1 course. I have an app, which reades `.txt` files and creates questions from them. `001.txt` structure is following:
```
X0100                           <- Shows the correct answer 1,2,3,4. In this case it's answer 2
1. What colour is sun?          <- Question line
Blue                            <- Answers below
Yellow
Green
Red
```

Initial plan:
- [ ] Read last file
- [ ] Convert to pd
- [ ] Create question for each word
    

In [2]:
import os
import pandas as pd
import random

In [3]:
# Method to read last data added
def get_latest_data_file():
    """
    Gets the most recently added file in the data directory

    Returns:
        str: Name of the most recently added file
    """
    data_dir = "data"
    files = []

    # Get all files in data directory with creation times
    for filename in os.listdir(data_dir):
        filepath = os.path.join(data_dir, filename)
        if os.path.isfile(filepath):
            creation_time = os.path.getctime(filepath)
            files.append((filename, creation_time))

    if not files:
        return None

    # Sort by creation time and get most recent
    latest_file = sorted(files, key=lambda x: x[1], reverse=True)[0][0]
    print(f"Latest file found: {latest_file}")
    return latest_file


In [5]:
latest_file = get_latest_data_file()
data_pd = pd.read_csv(
    f"data/{latest_file}", sep="=", header=None, names=["english", "japanese"]
)
print(data_pd.head())

Latest file found: demo.txt
     english             japanese
0  entrance              iriguchi
1   chimney               entotsu
2      roof                  yane
3   kitchen    kitchen, daidokoro
4      room                  heya


In [6]:
def generate_question_file(index, japanese_word, english_word, all_english_words):
    """
    Generates a single question file in the specified format

    Args:
        index (int): Question number (for filename)
        japanese_word (str): The Japanese word to test
        english_word (str): The correct English translation
        all_english_words (list): List of all English words to choose wrong answers from
    """
    # Create baza directory if it doesn't exist
    if not os.path.exists("baza"):
        os.makedirs("baza")

    # Get 3 random wrong answers
    wrong_answers = random.sample(
        [w for w in all_english_words if w != english_word], 3
    )

    # Generate filename with leading zeros
    filename = f"baza/{index:04d}.txt"

    # Combine correct and wrong answers
    answers = wrong_answers + [english_word]

    # Randomly shuffle answers and keep track of correct position
    correct_position = len(answers) - 1  # Initial position of correct answer
    random.shuffle(answers)
    correct_position = answers.index(
        english_word
    )  # New position after shuffle (0-based)

    # Create the correct answer marker (e.g., X1000, X0100, X0010, X0001)
    answer_marker = ["0"] * len(answers)
    answer_marker[correct_position] = "1"
    answer_marker = "X" + "".join(answer_marker)

    # Write to file
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"{answer_marker}\n")  # Mark correct answer
        f.write(f"{index}. {japanese_word}\n")  # Question
        for answer in answers:
            f.write(f"{answer}\n")  # Write each answer option

In [7]:
# Get all English words for wrong answers
all_english = data_pd["english"].tolist()

# Generate a file for each Japanese word
for index, row in data_pd.iterrows():
    generate_question_file(
        index + 1,  # Start from 1
        row["japanese"],
        row["english"],
        all_english,
    )

print(f"Generated {len(data_pd)} question files in the 'baza' folder")


Generated 5 question files in the 'baza' folder
